In [1]:
import geopandas as gpd
import pandas as pd
import networkx as nx
import osmnx as ox
import folium
import pickle
from shapely.geometry import box, Point

### Carregando o Grafo

In [2]:
# Ler o grafo de um arquivo usando pickle
with open('grafo_linhas.grafo', 'rb') as f:
    G = pickle.load(f)

In [3]:
print(len(G.nodes))

4544


In [4]:
print(G.nodes(data=True))
print(G.edges(data=True))


[(6856, {'coords': <POINT (-47.785 -15.763)>}), (7556, {'coords': <POINT (-47.789 -15.764)>}), (7558, {'coords': <POINT (-47.789 -15.77)>}), (7559, {'coords': <POINT (-47.792 -15.768)>}), (7560, {'coords': <POINT (-47.794 -15.764)>}), (7562, {'coords': <POINT (-47.789 -15.763)>}), (7563, {'coords': <POINT (-47.787 -15.763)>}), (6844, {'coords': <POINT (-47.786 -15.764)>}), (6134, {'coords': <POINT (-47.781 -15.779)>}), (2715, {'coords': <POINT (-47.779 -15.782)>}), (2397, {'coords': <POINT (-47.795 -15.803)>}), (2398, {'coords': <POINT (-47.801 -15.809)>}), (2399, {'coords': <POINT (-47.806 -15.813)>}), (2400, {'coords': <POINT (-47.806 -15.818)>}), (2401, {'coords': <POINT (-47.805 -15.821)>}), (2402, {'coords': <POINT (-47.807 -15.825)>}), (2403, {'coords': <POINT (-47.811 -15.829)>}), (2404, {'coords': <POINT (-47.814 -15.832)>}), (2405, {'coords': <POINT (-47.822 -15.835)>}), (2441, {'coords': <POINT (-47.826 -15.831)>}), (2535, {'coords': <POINT (-47.833 -15.819)>}), (3617, {'coor

### Receber 2 pontos (origem e destino) com suas respectivas coordenadas 

In [11]:
origem = (-15.989444964529529, -48.044418962814866) # Unb Gama
destino = (-15.79113644987054, -47.88317800514907)   # Conjunto Nacional
 
def calculaNoMaisPerto(G, long, lat):
    distancias = {}
    ponto = Point(long, lat)
    for node in G.nodes():
        node_coords = Point(G.nodes[node]['coords'])  
        dist = node_coords.distance(ponto)
        distancias[node] = {'dist': dist}
    # Find the nodes with the shortest distancias to the origin and destination
    noMaisPerto = min(distancias, key=lambda x: distancias[x]['dist'])
    return noMaisPerto

no_origem = calculaNoMaisPerto(G, origem[1], origem[0])
no_destino = calculaNoMaisPerto(G, destino[1], destino[0])

print(no_origem, no_destino)

7672 3482


### Encontar as paradas mais próximos em um raio de distância

In [16]:
import folium
from shapely.geometry import Point

raio = 0.007  # Raio desejado em metros ???

def calculaNosProximos(G, long, lat, raio):
    nos_proximos = []
    ponto = Point(long, lat)
    for node in G.nodes():
        node_coords = Point(G.nodes[node]['coords'])
        dist = node_coords.distance(ponto)
        if dist <= raio:
            nos_proximos.append(node)
    return nos_proximos

nos_origem = calculaNosProximos(G, origem[1], origem[0], raio)
nos_destino = calculaNosProximos(G, destino[1], destino[0], raio)

if nos_origem and nos_destino:
    print("Nós mais próximos dentro do raio de origem:", nos_origem)
    print("Nós mais próximos dentro do raio de destino:", nos_destino)

# Mapa folium
m = folium.Map(location=[origem[0], origem[1]], zoom_start=14)

# Marcadores para os nós próximos de origem
for node in nos_origem:
    coords = G.nodes[node]['coords']
    folium.Marker([coords.y, coords.x], icon=folium.Icon(color='blue')).add_to(m)

# Marcadores para os nós próximos de destino
for node in nos_destino:
    coords = G.nodes[node]['coords']
    folium.Marker([coords.y, coords.x], icon=folium.Icon(color='red')).add_to(m)

# Exiba o mapa
m

Nós mais próximos dentro do raio de origem: [7024, 7027, 1855, 1856, 7672, 7025, 7026]
Nós mais próximos dentro do raio de destino: [3250, 3251, 3480, 3482, 3483, 3484, 3368, 3369, 3267, 3268]


### Verificando com o Folium

In [13]:
# # Inicializar o mapa do Folium (ajuste as coordenadas iniciais conforme necessário)
m = folium.Map(location=[-15.7942, -47.8825], zoom_start=12)

# Adicionar as paradas como marcadores no mapa

lat1, long1 = G.nodes[no_origem]['coords'].y, G.nodes[no_origem]['coords'].x
lat2, long2 = G.nodes[no_destino]['coords'].y, G.nodes[no_destino]['coords'].x

folium.Marker([lat1, long1], popup=f'Parada {no_origem}').add_to(m)
folium.Marker([lat2, long2], popup=f'Parada {no_destino}').add_to(m)

m

### Algoritmo rota ótima

- Deve receber o nó origem e o nó destino do Grafo
- Percorrer o Grafo usando algoritmo de Dijkstra, cujo o peso é a distancia entre as arestas
- Armazenar as Paradas percorridas e as linhas de ônibus necessárias para chegar da origem ao destino

In [14]:
# Calcula rota otima com algoritmo de Dijkstra
rota_otima = nx.shortest_path(G, no_origem, no_destino, weight="dist", method="dijkstra")

print(rota_otima)

[7672, 2023, 7021, 7020, 7015, 7014, 7010, 7006, 2620, 3369, 3464, 3480, 3482]


In [17]:
# # Inicializar o mapa do Folium (ajuste as coordenadas iniciais conforme necessário)
m = folium.Map(location=[-15.7942, -47.8825], zoom_start=12)

linhas = []

# Cores para os grafos
cores = ['blue', 'red', 'yellow', 'green']
contador_cor = 0
cor = cores[contador_cor] 

# Adicionando primeira parada no Folium
lat, lon = G.nodes[rota_otima[0]]['coords'].y, G.nodes[rota_otima[0]]['coords'].x
folium.Marker([lat, lon], popup=f'Parada {rota_otima[0]}').add_to(m)

for i in range(1, len(rota_otima)):

    parada_origem = rota_otima[i-1] 
    parada_destino = rota_otima[i]

    # Adicionando paradas no Folium
    lat2, lon2 = G.nodes[parada_destino]['coords'].y, G.nodes[parada_destino]['coords'].x
    folium.Marker([lat2, lon2], popup=f'Parada {parada_destino}').add_to(m)

    # Adicionando arestas no Folium
    origem_coords = (G.nodes[parada_origem]['coords'].y, G.nodes[parada_origem]['coords'].x)
    destino_coords = (lat2, lon2)
    dados_aresta = G.get_edge_data(parada_origem, parada_destino)

    # Adicionando nova linha na lista
    if dados_aresta['linha'] not in linhas:
        linhas.append(dados_aresta['linha'])
        contador_cor += 1
        cor = cores[contador_cor % len(cores)]

    folium.PolyLine([origem_coords, destino_coords], tooltip=dados_aresta['linha'], color=cor, weight=2).add_to(m)

        
print(linhas)

[0.224, 2210.0, 2207.0, 2206.0, 0.023, 501.4]


In [18]:
m